## Dev env configuration

In [1]:
# install the required python packages
# !pip install numpy pandas yfinance matplotlib mplfinance seaborn

## Imports

In [2]:
# import the necessary packages
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import yfinance as yf

## Configuring our date range and ticker

In [3]:
# set the start and end dates for our market data request to be TTM
end_date = datetime(year=2025, month=3, day=1)
start_date = end_date - timedelta(days=365)

# set the name of the ticker we want to download market data for
ticker = ["NVDA", "AAPL"]
# ticker = "AAPL"

## Computing daily 50 and 200 moving averages

In [29]:
# download daily market data
df_daily = yf.download(
    tickers=ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    auto_adjust=True,
    progress=False
)
df_daily

Price            Close                    High                     Low  \
Ticker            AAPL        NVDA        AAPL        NVDA        AAPL   
Date                                                                     
2024-03-01  178.815674   82.255676  179.681580   82.276674  176.546390   
2024-03-04  174.277115   85.212837  176.068644   87.670141  172.973259   
2024-03-05  169.320496   85.943665  171.231471   86.076636  168.822845   
2024-03-06  168.325180   88.679016  170.435227   89.702776  167.887245   
2024-03-07  168.205750   92.647079  169.927615   92.745056  167.698152   
...                ...         ...         ...         ...         ...   
2025-02-24  247.100006  130.279999  248.860001  138.589996  244.419998   
2025-02-25  247.039993  126.629997  250.000000  130.199997  244.910004   
2025-02-26  240.360001  131.279999  244.979996  133.729996  239.130005   
2025-02-27  237.300003  120.150002  242.460007  135.009995  237.059998   
2025-02-28  241.839996  124.919998  242.089996  125.089996  230.199997   

Price                         Open                Volume             
Ticker            NVDA        AAPL        NVDA      AAPL       NVDA  
Date                                                                 
2024-03-01   79.412481  178.706190   79.977323  73488000  479135000  
2024-03-04   83.695270  175.322168   84.106149  81510100  615616000  
2024-03-05   83.397270  169.957487   85.249828  95132400  520639000  
2024-03-06   87.009413  170.256065   88.001183  68587700  582520000  
2024-03-07   89.580804  168.355038   90.136672  71765100  608119000  
...                ...         ...         ...       ...        ...  
2025-02-24  130.080002  244.929993  136.559998  51326400  251381100  
2025-02-25  124.440002  248.000000  129.979996  48013300  271428700  
2025-02-26  128.490005  244.330002  129.990005  44433600  322553800  
2025-02-27  120.010002  239.410004  135.000000  41153600  443175800  
2025-02-28  116.400002  236.949997  118.019997  56796200  388160100  

[250 rows x 10 columns]

In [27]:
df_daily[:, "50MA"] = df_daily.xs("Close", axis=1, level="Price").rolling(window=50).mean()

TypeError: unhashable type: 'slice'

In [28]:
# calculate the daily 50 and 200 MAs
df_daily[:, "50MA"] = df_daily["Close"].groupby("Ticker").transform(lambda x: x.rolling(window=50).mean())
df_daily

KeyError: 'Ticker'

In [ ]:
df_daily["AAPL"]

In [ ]:
# # construct additional plots for the MAs
# addt_plots = [
#     mpf.make_addplot(df_daily["50MA"], color="blue", width=1.5, label="50MA"),
#     mpf.make_addplot(df_daily["200MA"], color="red", width=1.5, label="200MA"),
# ]

# # plot the daily OHLCV and MAs
# mpf.plot(
#     df_daily
    
# )

## Computing weekly 10 and 40 moving averages

## Compututing monthly 10 moving average